In [1]:
import pandas as pd
import numpy as np
import datetime

from sklearn.neighbors import LocalOutlierFactor
from pandas_profiling import ProfileReport

KeyboardInterrupt: 

In [2]:
data_path = '../data/'
output_path = '../out/'

In [3]:
data = pd.read_csv(data_path + 'bikes.csv')

/home/marek-szyd/Programs/anaconda3/envs/dv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (2,4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
data.drop(
    [
        'departure_latitude', 'departure_longitude', 'return_latitude', 'return_longitude',
        'departure_id', 'return_id'

    ],
    axis=1, inplace=True
)

In [5]:
data.head()

,departure,return,departure_name,return_name,distance (m),duration (sec.),avg_speed (km/h),Air temperature (degC)
0,2020-03-23 06:09:44,2020-03-23 06:16:26,Kuusitie,Esterinportti,1747.0,401.0,0.261397,0.9
1,2020-03-23 06:11:58,2020-03-23 06:26:31,Kamppi (M),Kasarmitori,1447.0,869.0,0.099908,0.9
2,2020-03-23 06:16:29,2020-03-23 06:24:23,Porolahden koulu,Agnetankuja,1772.0,469.0,0.226695,0.9
3,2020-03-23 06:33:53,2020-03-23 07:14:03,Vallipolku,Korppaanmäentie,7456.0,2406.0,0.185935,0.9
4,2020-03-23 06:36:09,2020-03-23 07:04:10,Länsisatamankatu,Vilhonvuorenkatu,7120.0,1679.0,0.254437,0.9


In [6]:
data.describe()

,distance (m),duration (sec.),avg_speed (km/h),Air temperature (degC)
count,1.215746e+07,1.215746e+07,1.215391e+07,1.214156e+07
mean,2.295275e+03,9.597751e+02,3.355556e-01,1.565044e+01
std,2.452067e+04,7.346528e+03,3.428006e+01,5.497952e+00
min,-4.292467e+06,0.000000e+00,-4.689001e+02,-5.200000e+00
25%,1.000000e+03,3.440000e+02,1.467403e-01,1.230000e+01
50%,1.739000e+03,5.860000e+02,1.863679e-01,1.640000e+01
75%,2.869000e+03,9.710000e+02,2.204348e-01,1.930000e+01
max,3.681399e+06,5.401659e+06,1.699104e+04,3.290000e+01


In [7]:
data.columns

Index(['departure', 'return', 'departure_name', 'return_name', 'distance (m)',
       'duration (sec.)', 'avg_speed (km/h)', 'Air temperature (degC)'],
      dtype='object')

In [8]:
data.rename(columns={
    'distance (m)' : 'distance',
    'duration (sec.)' : 'duration',
    'avg_speed (km/h)' : 'avg_speed',
    'Air temperature (degC)' : 'temperature'
}, inplace=True)

In [9]:
data.columns

Index(['departure', 'return', 'departure_name', 'return_name', 'distance',
       'duration', 'avg_speed', 'temperature'],
      dtype='object')

In [10]:
data.dtypes

departure          object
return             object
departure_name     object
return_name        object
distance          float64
duration          float64
avg_speed         float64
temperature       float64
dtype: object

In [11]:
data['departure'] = pd.to_datetime(data['departure'])
data['return'] = pd.to_datetime(data['return'])

In [12]:
data.dtypes

departure         datetime64[ns]
return            datetime64[ns]
departure_name            object
return_name               object
distance                 float64
duration                 float64
avg_speed                float64
temperature              float64
dtype: object

In [13]:
data.head(10)

,departure,return,departure_name,return_name,distance,duration,avg_speed,temperature
0,2020-03-23 06:09:44,2020-03-23 06:16:26,Kuusitie,Esterinportti,1747.0,401.0,0.261397,0.9
1,2020-03-23 06:11:58,2020-03-23 06:26:31,Kamppi (M),Kasarmitori,1447.0,869.0,0.099908,0.9
2,2020-03-23 06:16:29,2020-03-23 06:24:23,Porolahden koulu,Agnetankuja,1772.0,469.0,0.226695,0.9
3,2020-03-23 06:33:53,2020-03-23 07:14:03,Vallipolku,Korppaanmäentie,7456.0,2406.0,0.185935,0.9
4,2020-03-23 06:36:09,2020-03-23 07:04:10,Länsisatamankatu,Vilhonvuorenkatu,7120.0,1679.0,0.254437,0.9
5,2020-03-23 06:37:52,2020-03-23 06:58:56,Radiokatu,Porthania,5169.0,1262.0,0.245753,0.9
6,2020-03-23 06:39:51,2020-03-23 06:45:30,Tyynenmerenkatu,Hietalahdentori,1194.0,335.0,0.213851,0.9
7,2020-03-23 06:44:37,2020-03-24 12:02:19,Itämerentori,Meilahden sairaala,3651.0,1195.0,0.183314,0.9
8,2020-03-23 06:47:18,2020-03-23 07:01:09,Koskelantie,Kalasatama (M),3120.0,827.0,0.226360,0.9
9,2020-03-23 06:47:22,2020-03-23 06:56:22,Rautatieläisenkatu,A.I. Virtasen aukio,2070.0,535.0,0.232150,0.9


## Drop data with uselss data - distance == 0, etc.

In [14]:
data.drop(data[data['distance'] == 0].index, inplace=True)


## Create HTML report of the dataset in out/ directory

In [15]:
# !!! Full profiling takes a lot of time and big chunk of RAM; Uncomment the next line to run full profiler.

# profile = ProfileReport(data, title='Helsinki City Bikes')
profile = ProfileReport(data, title='Helsinki City Bikes', minimal=True)
profile.to_file(output_path + 'bikes.html')

In [16]:
data.groupby(by='departure_name').size()

departure_name
A.I. Virtasen aukio                       10605
Aalto-yliopisto (M), Korkeakouluaukio     92852
Aalto-yliopisto (M), Tietotie             25808
Abraham Wetterin tie                      11088
Agnetankuja                               13412
                                          ...  
Von Daehnin katu                           3292
Westendinasema                            15426
Westendintie                               6932
Yhdyskunnankuja                            8422
Ympyrätalo                               167796
Length: 347, dtype: int64

In [17]:
data[((data.distance <= 0) | (data.avg_speed <= 0))]

,departure,return,departure_name,return_name,distance,duration,avg_speed,temperature
1456090,2020-07-04 12:26:49,2020-07-04 12:49:33,Verkatehtaanpuisto,Lepolantie,-4289957.0,1362.0,-188.984890,16.4
1669654,2020-07-17 17:05:32,2020-07-17 17:33:08,Valimotie,Jämeräntaival,-4286859.0,1654.0,-155.508791,23.6
1783328,2020-07-23 21:36:33,2020-07-23 21:47:05,Sörnäinen (M),Intiankatu,-4292467.0,633.0,-406.868910,13.5
1912295,2020-07-31 23:45:28,2020-08-01 00:09:21,Pohjankulma,Lehtisaarentie,-4291050.0,1430.0,-180.044056,15.5
2294615,2020-08-20 16:08:40,2020-08-20 16:29:36,Painiitty,Teljäntie,-4290278.0,1255.0,-205.112892,21.8
6888855,2019-10-27 20:13:52,2019-10-27 20:23:02,Niittymaa,Länsituuli,-4290436.0,549.0,-468.900109,1.9
10027503,2018-10-15 19:01:14,2018-10-15 19:16:20,Velodrominrinne,Hakaniemi (M),-4286564.0,905.0,-284.192088,12.0


In [23]:
clf = LocalOutlierFactor(n_neighbors=20)
outlier_array = clf.fit_predict(data[['distance', 'duration', 'avg_speed']])

In [28]:
# Map -1 to True and 1 to False for outliers
converter = lambda val: True if val == -1 else False
outliers = np.array([converter(val) for val in outlier_array])

In [25]:
dat_outliers = data[outliers]
print(f"Detected {len(dat_outliers)} outliers in dataset!")
dat_outliers.head(20)

ValueError: Item wrong length 100 instead of 11769724.